In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo

# --- Carregar e preparar os dados ---
# fetch dataset 
student_performance = fetch_ucirepo(id=320) 
  
# data (as pandas dataframes) 
X = student_performance.data.features 
y = student_performance.data.targets 
  
# metadata 
print(student_performance.metadata) 
  
# variable information 
print(student_performance.variables) 

#Analise Tecnica
print(f"Amostras: {X.shape[0]}, Variáveis: {X.shape[1]}")
display(X.head())
display(y.head())

# Tratar dados ausentes
if X.isnull().values.any():
    X = X.fillna(X.mean(numeric_only=True))
    X = X.fillna(X.mode().iloc[0])
else:
    print(" Sem dados em falta.")

# Codificação one-hot
X = pd.get_dummies(X, drop_first=True)

# Padronização
num_cols = X.select_dtypes(include=[np.number]).columns
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# --- Regressão Linear ---
X_train, X_test, y_train, y_test = train_test_split(X, y['G3'], test_size=0.2, random_state=42)
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
y_pred_lin = lin_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_lin)
print(f"Erro Quadrático Médio (MSE): {mse:.2f}")

# --- Regressão Logística ---
y_bin = (y['G3'] >= 10).astype(int)
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, y_bin, test_size=0.2, random_state=42)
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_log, y_train_log)
y_pred_log = log_model.predict(X_test_log)

print("Matriz de Confusão:")
print(confusion_matrix(y_test_log, y_pred_log))
print("Acurácia:", accuracy_score(y_test_log, y_pred_log))
print("Relatório de Classificação:")
print(classification_report(y_test_log, y_pred_log))

# --- Gráficos ---
# Matriz de Confusão
cm = confusion_matrix(y_test_log, y_pred_log)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Reprovado", "Aprovado"], yticklabels=["Reprovado", "Aprovado"])
plt.title("Matriz de Confusão")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.show()

# Regressão Linear - Real vs Previsto
plt.figure(figsize=(6,4))
plt.scatter(y_test, y_pred_lin, color='purple', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Valor Real (G3)")
plt.ylabel("Valor Previsto (G3)")
plt.title("Regressão Linear - Real vs Previsto")
plt.grid(True)
plt.show()

# Combinar X e y['G3'] para formar um DataFrame completo
df = X.copy()
df['G3'] = y['G3']

# Calcular a correlação com G3 (ignorando a própria G3)
correlation_with_g3 = df.corr(numeric_only=True)['G3'].drop('G3').sort_values(key=abs, ascending=False)

# Gráfico de barras horizontal
plt.figure(figsize=(10, 6))
correlation_with_g3.plot(kind='barh', color='steelblue')
plt.axvline(0, color='black', linewidth=0.8)
plt.title('Correlação das Variáveis com G3')
plt.xlabel('Coeficiente de Correlação')
plt.gca().invert_yaxis()  # Mostra as maiores correlações no topo
plt.grid(axis='x')
plt.tight_layout()
plt.show()

# Gráfico de Densidade de G3
plt.figure(figsize=(8, 5))
sns.kdeplot(y['G3'], shade=True, color='darkgreen')
plt.title("Distribuição da Nota Final (G3)")
plt.xlabel("Nota G3")
plt.ylabel("Densidade")
plt.grid(True)
plt.show()


#Grafico de Barras da media da nota final por escolaridade da mãe
plt.figure(figsize=(8, 5))
sns.barplot(data=student_performance.data.original, x='Medu', y=y['G3'])
plt.xlabel("Escolaridade da Mãe (0 = nenhuma, 4 = ensino superior)")
plt.ylabel("Média da Nota Final (G3)")
plt.title("Média da Nota Final por Escolaridade da Mãe")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

#Grafico de Faltas com o nivel de alcool ao fim de semana
plt.figure(figsize=(8, 5))
sns.scatterplot(
    data=student_performance.data.original,
    x='absences',
    y=y['G3'],
    hue='Walc',  # consumo de álcool nos fins de semana
    palette='coolwarm'
)
plt.xlabel("Faltas (absences)")
plt.ylabel("Nota Final (G3)")
plt.title("Faltas vs Nota Final com Nível de Álcool no Fim de Semana")
plt.legend(title="Álcool (Walc)", loc='upper right')
plt.tight_layout()
plt.show()

#Grafico Nota Final G3 por Tempo de Estudo e Sexo
df = student_performance.data.original.copy()
df['sexo'] = df['sex'].map({'M': 'Masculino', 'F': 'Feminino'})

plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='studytime', y='G3', hue='sexo', palette='coolwarm')
plt.title("Nota Final G3 por Tempo de Estudo e Sexo")
plt.xlabel("Tempo de Estudo (1=baixo, 4=alto)")
plt.ylabel("Nota Final (G3)")
plt.grid(True)
plt.tight_layout()
plt.show()

#Grafico de Correlação entre Notas Intermédias e Nota Final
plt.figure(figsize=(8, 5))
sns.scatterplot(x=student_performance.data.original['G1'], y=y['G3'], label='G1 vs G3', alpha=0.6)
sns.scatterplot(x=student_performance.data.original['G2'], y=y['G3'], label='G2 vs G3', alpha=0.6)
plt.title("Correlação entre Notas Intermédias e Nota Final")
plt.xlabel("Nota G1 / G2")
plt.ylabel("Nota Final G3")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()